In [3]:
%pip --version

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "python"



In [2]:
%pip install keyboard

   ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
   ---------------------------------------- 58.1/58.1 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timezone
from bs4 import BeautifulSoup

import time
import json
import os
import logging
import requests
from urllib.parse import urlparse

In [4]:
# MongoDB 연결 설정을 전역으로 이동
client = MongoClient('mongodb://localhost:27017/')
db = client['property_db']
collection = db['properties']

In [3]:
def save_image_info_to_mongodb(property_id, category, img_url, file_path, category_count):
    """이미지 정보를 단지 수준에서 MongoDB에 저장"""
    try:
        complexes_collection = db['complexes']
        complex_data = complexes_collection.find_one({"매물목록.매물번호": property_id})
        
        if not complex_data:
            print("해당 매물이 속한 단지 정보를 찾을 수 없습니다.")
            return
            
        # 이미지 정보 추가 (카테고리와 순번이 같은 이미지가 없을 때만)
        result = complexes_collection.update_one(
            {
                "단지명": complex_data['단지명'],
                "이미지목록": {
                    "$not": {
                        "$elemMatch": {
                            "카테고리": category,
                            "카테고리순번": category_count
                        }
                    }
                }
            },
            {"$push": {"이미지목록": {
                "카테고리": category,
                "이미지URL": img_url,
                "저장경로": file_path,
                "카테고리순번": category_count
            }}}
        )
        
        print(f"이미지 저장 완료: {complex_data['단지명']} - {category}_{category_count}")
            
    except Exception as e:
        print(f"이미지 정보 저장 중 오류 발생: {str(e)}")

In [280]:
# 이미지 저장할 경로 지정
save_path = r"C:\Users\pqstv\Desktop\real_estate_automization\images"  # 원하는 경로로 변경하세요

In [17]:
def download_images(driver, property_id, path):
    # complexes 컬렉션에서 매물 정보와 단지명 찾기
    # complexes_collection = db['complexes']
    # complex_data = complexes_collection.find_one({"매물목록.매물번호": property_id})
        
    # if not complex_data:
    #     print("매물 정보를 찾을 수 없습니다.")
    #     return
            
    # complex_name = complex_data['단지명']
        
    # # 이미 이미지가 있는지 확인
    # if complex_data.get('이미지목록', []):
    #     print(f"단지 '{complex_name}'의 이미지가 이미 존재합니다. 다운로드를 건너뜁니다.")
    #     return
    
    main_window = driver.window_handles[-1]
    # 사진 다운로드 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="summaryInfo"]/div[2]/div[2]/button[1]').click()
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div/button[1]').click()
    time.sleep(2)
    
    # 새로 열린 팝업 창으로 전환
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    
    try:
        # 이미 찾은 단지명 사용
        complex_name = ''.join(e for e in complex_name if e.isalnum() or e.isspace())
        complex_name = complex_name.replace(' ', '_')
        
        # 이미지 요소들이 있는 li 태그들 찾기
        image_items = driver.find_elements(By.CSS_SELECTOR, "ul.lst_view li._js_thumbnail_image_item_view")
        
        if not image_items:
            print(f"매물 {property_id}의 이미지가 없습니다.")
            return
        
        save_path = f"{path}/{complex_name}"
        
        # 저장 경로가 없으면 생성
        os.makedirs(save_path, exist_ok=True)
        
        # 카테고리별 카운터 초기화
        category_counters = {}
        
        # 각 이미지 다운로드
        for item in image_items:
            try:
                category = item.get_attribute('_categorycode')
                img = item.find_element(By.TAG_NAME, 'img')
                img_url = img.get_attribute('src')
                img_url = img_url.replace('?type=m70', '')
                
                if category not in category_counters:
                    category_counters[category] = 1
                else:
                    category_counters[category] += 1
                
                response = requests.get(img_url)
                if response.status_code == 200:
                    file_name = f"{category}_{category_counters[category]}.jpg"
                    file_path = os.path.join(save_path, file_name)
                    
                    with open(file_path, 'wb') as f:
                        f.write(response.content)
                    print(f"이미지 다운로드 완료: {file_name}")
                    
                    # MongoDB에 이미지 정보 저장
                    save_image_info_to_mongodb(
                        property_id=property_id,
                        category=category,
                        img_url=img_url,
                        file_path=file_path,
                        category_count=category_counters[category]
                    )
                else:
                    print(f"이미지 다운로드 실패 ({complex_name}_{category}_{category_counters[category]}): HTTP {response.status_code}")
            
            except Exception as e:
                print(f"이미지 처리 중 오류 발생: {str(e)}")
                continue
        
        print(f"매물 {property_id}의 모든 이미지 다운로드 완료")
        print("카테고리별 이미지 수:", category_counters)
        
        driver.close()
                    
        # 원래 창으로 돌아가기
        driver.switch_to.window(main_window)
        time.sleep(1)
        
    except Exception as e:
        print(f"이미지 다운로드 중 오류 발생: {str(e)}")

In [282]:
# 아파트, 재건축, 주상복합인 경우
def extract_and_save_apt(driver, property_id):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 매물 정보 딕셔너리 생성
    property_data = {
        "생성일자": datetime.now(),
        "매물번호": property_id,
    }
    
    # 단지명, 거래종류, 가격 정보 추출
    try:
        property_data['단지명'] = driver.find_element(By.XPATH, '//*[@id="complexTitle"]').text.strip()
        property_data['세부단지명'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/h4/strong').text.strip()
        property_data['거래종류'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[1]').text.strip()
        property_data['가격'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text.strip()
    except Exception as e:
        print(f"기본 정보 추출 실패: {str(e)}")
    
    # 추출할 정보 매핑
    property_fields = {
        "매물특징": "매물특징",
        "공급/전용면적": "공급/전용면적",
        "해당층/총층": "해당층/총층",
        "방수/욕실수": "방수/욕실수",
        "방향": "방향",
        "현관구조": "현관구조",
        "난방(방식/연료)": "난방(방식/연료)",
        "입주가능일": "입주가능일",
        "총주차대수": "총주차대수",
        "해당면적세대수": "해당면적세대수",
        "건축물용도": "건축물용도",
        "매물설명": "매물설명"
    }
    
    # 각 필드 정보 추출
    for key, field in property_fields.items():
        try:
            field_row = soup.find('th', string=field)
            if field_row:
                field_value = field_row.find_next('td').text.strip()
                property_data[key] = field_value
        except Exception as e:
            print(f"{field} 정보 추출 실패: {str(e)}")
            property_data[key] = None
    
    try:
        complex_name = property_data['단지명']
        complexes_collection = db['complexes']
        
        # 먼저 같은 매물번호가 있는지 확인
        existing_complex = complexes_collection.find_one({
            "단지명": complex_name,
            "매물목록.매물번호": property_id
        })
        
        if existing_complex:
            # 같은 매물번호가 있으면 해당 매물 정보만 업데이트
            result = complexes_collection.update_one(
                {
                    "단지명": complex_name,
                    "매물목록.매물번호": property_id
                },
                {"$set": {"매물목록.$": property_data}}
            )
            print(f"기존 매물 정보 업데이트: {complex_name} - {property_id}")
        else:
            # 새로운 매물이면 추가
            result = complexes_collection.update_one(
                {"단지명": complex_name},
                {
                    "$push": {"매물목록": property_data},
                    "$setOnInsert": {"이미지목록": []}
                },
                upsert=True
            )
            print(f"새로운 매물 추가: {complex_name} - {property_id}")
        
        return property_data
        
    except Exception as e:
        print(f"매물 정보 저장 중 오류 발생: {str(e)}")
        return None

In [6]:
def extract_and_save_market(driver, property_id):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 매물 정보 딕셔너리 생성
    property_data = {
        "생성일자": datetime.now(),
        "매물번호": property_id,
    }
    
    # 단지명, 거래종류, 가격 정보 추출
    try:
        property_data['단지명'] = driver.find_element(By.XPATH, '//*[@id="complexTitle"]').text.strip()
        property_data['세부단지명'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/h4/strong').text.strip()
        property_data['거래종류'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[1]').text.strip()
        property_data['가격'] = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]').text.strip()
    except Exception as e:
        print(f"기본 정보 추출 실패: {str(e)}")
    
    # 추출할 정보 매핑
    property_fields = {
        "소재지": "소재지",
        "매물특징": "매물특징",
        "계약/전용면적": "계약/전용면적",
        "해당층/총층": "해당층/총층",
        "입주가능일": "입주가능일",
        "월관리비": "월관리비",
        "방향": "방향",
        "현재업종": "현재업종",
        "추천업종": "추천업종",
        "주차가능여부": "주차가능여부",
        "총주차대수": "총주차대수",
        "난방(방식/연료)": "난방(방식/연료)",
        "해당면적세대수": "해당면적세대수",
        "건축물용도": "건축물용도",
        "매물설명": "매물설명"
    }
    
    # 각 필드 정보 추출
    for key, field in property_fields.items():
        try:
            field_row = soup.find('th', string=field)
            if field_row:
                field_value = field_row.find_next('td').text.strip()
                property_data[key] = field_value
        except Exception as e:
            print(f"{field} 정보 추출 실패: {str(e)}")
            property_data[key] = None
    
    try:
        complex_name = property_data['단지명']
        commercial_collection = db['commercial']
        
        # 먼저 같은 매물번호가 있는지 확인
        existing_complex = commercial_collection.find_one({
            "단지명": complex_name,
            "매물목록.매물번호": property_id
        })
        
        if existing_complex:
            # 같은 매물번호가 있으면 해당 매물 정보만 업데이트
            result = commercial_collection.update_one(
                {
                    "단지명": complex_name,
                    "매물목록.매물번호": property_id
                },
                {"$set": {"매물목록.$": property_data}}
            )
            print(f"기존 매물 정보 업데이트: {complex_name} - {property_id}")
        else:
            # 새로운 매물이면 추가
            result = commercial_collection.update_one(
                {"단지명": complex_name},
                {
                    "$push": {"매물목록": property_data},
                    "$setOnInsert": {"이미지목록": []}
                },
                upsert=True
            )
            print(f"새로운 매물 추가: {complex_name} - {property_id}")
        
        return property_data
        
    except Exception as e:
        print(f"매물 정보 저장 중 오류 발생: {str(e)}")
        return None

In [190]:
# 빌라인 경우
def extract_and_save_villa(driver, property_id):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 추출할 정보 매핑
    property_fields = {
        "매물번호": property_id,
        "소재지": "소재지",
        "거래종류": "거래종류",
        "매매가": "매매가",
        "공급면적": "공급면적",
        "전용면적": "전용면적",
        "해당층/총층": "해당층/총층",
        "입주가능일": "입주가능일",
        "계약기간": "계약기간",
        "매물특징": "매물특징",
        "상세설명": "상세설명"
    }
    
    # MongoDB에 저장할 데이터 딕셔너리 생성
    property_data = {
        "생성일자": datetime.now(),
        "매물번호": property_id,
        "이미지경로": f"{save_path}/{property_id}"  # 이미지 저장 경로도 추가
    }
    
    # 각 필드 정보 추출
    for key, field in property_fields.items():
        try:
            field_row = soup.find('th', string=field)
            if field_row:
                field_value = field_row.find_next('td').text.strip()
                property_data[key] = field_value
        except Exception as e:
            print(f"{field} 정보 추출 실패: {str(e)}")
            property_data[key] = None
    
    try:        
        # 기존 데이터가 있는지 확인 (매물번호로 검색)
        existing_data = collection.find_one({"매물번호": property_id})
        
        if existing_data:
            # 업데이트
            result = collection.update_one(
                {"매물번호": property_id},
                {"$set": property_data}
            )
            print(f"MongoDB 데이터 업데이트 완료: {property_id}")
        else:
            # 새로운 데이터 삽입
            result = collection.insert_one(property_data)
            print(f"MongoDB에 새 데이터 저장 완료: {result.inserted_id}")
        
        return property_data
        
    except Exception as e:
        print(f"MongoDB 저장 중 오류 발생: {str(e)}")
        return None
        
    return None

In [191]:
# 오피스텔인 경우
def extract_and_save_officetel(driver, property_id):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # 추출할 정보 매핑
    property_fields = {
        "매물번호": property_id,
        "소재지": "소재지",
        "거래종류": "거래종류",
        "매매가": "매매가",
        "공급면적": "공급면적",
        "전용면적": "전용면적",
        "해당층/총층": "해당층/총층",
        "입주가능일": "입주가능일",
        "계약기간": "계약기간",
        "매물특징": "매물특징",
        "상세설명": "상세설명"
    }
    
    # MongoDB에 저장할 데이터 딕셔너리 생성
    property_data = {
        "생성일자": datetime.now(),
        "매물번호": property_id,
        "이미지경로": f"{save_path}/{property_id}"  # 이미지 저장 경로도 추가
    }
    
    # 각 필드 정보 추출
    for key, field in property_fields.items():
        try:
            field_row = soup.find('th', string=field)
            if field_row:
                field_value = field_row.find_next('td').text.strip()
                property_data[key] = field_value
        except Exception as e:
            print(f"{field} 정보 추출 실패: {str(e)}")
            property_data[key] = None
    
    try:        
        # 기존 데이터가 있는지 확인 (매물번호로 검색)
        existing_data = collection.find_one({"매물번호": property_id})
        
        if existing_data:
            # 업데이트
            result = collection.update_one(
                {"매물번호": property_id},
                {"$set": property_data}
            )
            print(f"MongoDB 데이터 업데이트 완료: {property_id}")
        else:
            # 새로운 데이터 삽입
            result = collection.insert_one(property_data)
            print(f"MongoDB에 새 데이터 저장 완료: {result.inserted_id}")
        
        return property_data
        
    except Exception as e:
        print(f"MongoDB 저장 중 오류 발생: {str(e)}")
        return None
        
    return None

In [192]:
# 사무실인 경우
def extract_and_save_office(driver, property_id):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 추출할 정보 매핑
    property_fields = {
        "매물번호": property_id,
        "소재지": "소재지",
        "거래종류": "거래종류",
        "매매가": "매매가",
        "공급면적": "공급면적",
        "전용면적": "전용면적",
        "해당층/총층": "해당층/총층",
        "입주가능일": "입주가능일",
        "계약기간": "계약기간",
        "매물특징": "매물특징",
        "상세설명": "상세설명"
    }
    
    # MongoDB에 저장할 데이터 딕셔너리 생성
    property_data = {
        "생성일자": datetime.now(),
        "매물번호": property_id,
        "이미지경로": f"{save_path}/{property_id}"  # 이미지 저장 경로도 추가
    }
    
    # 각 필드 정보 추출
    for key, field in property_fields.items():
        try:
            field_row = soup.find('th', string=field)
            if field_row:
                field_value = field_row.find_next('td').text.strip()
                property_data[key] = field_value
        except Exception as e:
            print(f"{field} 정보 추출 실패: {str(e)}")
            property_data[key] = None
    
    try:        
        # 기존 데이터가 있는지 확인 (매물번호로 검색)
        existing_data = collection.find_one({"매물번호": property_id})
        
        if existing_data:
            # 업데이트
            result = collection.update_one(
                {"매물번호": property_id},
                {"$set": property_data}
            )
            print(f"MongoDB 데이터 업데이트 완료: {property_id}")
        else:
            # 새로운 데이터 삽입
            result = collection.insert_one(property_data)
            print(f"MongoDB에 새 데이터 저장 완료: {result.inserted_id}")
        
        return property_data
        
    except Exception as e:
        print(f"MongoDB 저장 중 오류 발생: {str(e)}")
        return None
        
    return None

In [2]:
def check_property_type(driver):
    """매물 종류 확인"""
    valid_types = ['아파트', '재건축', '오피스텔', '상가', '빌라', '사무실']
    xpaths = [
        '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[1]/span[2]',
        '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[1]/span[3]'
    ]
    
    try:
        # 1. 페이지가 완전히 로드될 때까지 좀 더 기다리기
        time.sleep(3)
        
        for xpath in xpaths:
            try:
                # 2. JavaScript로 요소 찾기 시도
                element = driver.execute_script(
                    f"return document.evaluate('{xpath}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;"
                )
                
                if element:
                    property_type = element.text.strip()
                    print(f"JS로 찾은 property_type: {property_type}")
                    
                    if property_type in valid_types:
                        return property_type
                        
                # 3. 다른 방법으로도 시도
                element = driver.find_element(By.XPATH, xpath)
                property_type = element.text.strip()
                print(f"일반적인 방법으로 찾은 property_type: {property_type}")
                
                if property_type in valid_types:
                    return property_type
                    
            except Exception as e:
                print(f"시도 실패: {str(e)}")
                continue
                
        print("유효한 매물 종류를 찾을 수 없습니다")
        return None
        
    except Exception as e:
        print(f"전체 프로세스 오류: {str(e)}")
        return None

In [284]:
def process_property_info(driver, property_id):
    """매물 종류에 따라 적절한 함수 실행"""
    try:
        property_type = check_property_type(driver)
        
        if property_type in ["아파트", "재건축", "주상복합"]:
            print("아파트 매물 정보 추출 시작")
            return extract_and_save_apt(driver, property_id)
        elif property_type == "상가":
            print("상가 매물 정보 추출 시작")
            return extract_and_save_market(driver, property_id)
        elif property_type == "빌라":
            print("빌라 매물 정보 추출 시작")
            return extract_and_save_villa(driver, property_id)
        elif property_type == "오피스텔":
            print("오피스텔 매물 정보 추출 시작")
            return extract_and_save_officetel(driver, property_id)
        elif property_type == "사무실":
            print("사무실 매물 정보 추출 시작")
            return extract_and_save_office(driver, property_id)
        else:
            print(f"처리할 수 없는 매물 종류입니다: {property_type}")
            return None
            
    except Exception as e:
        print(f"매물 정보 처리 중 오류 발생: {str(e)}")
        return None

In [5]:
chrome_options = Options()
service = Service(r'C:\Users\pqstv\tmttd\MARS\BHL\services\crawling\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)

In [6]:
# 부동산 뱅크 로그인 페이지 이동
driver.get('https://www.neonet.co.kr/novo-rebank/view/member/MemberLogin.neo?login_check=yes&return_url=/novo-rebank/index.neo')

In [7]:
# 로그인
id_input = driver.find_element(By.XPATH, '//*[@id="input_id"]')
id_input.send_keys('5164980')
pwd_input = driver.find_element(By.XPATH, '//*[@id="input_pw"]')
pwd_input.send_keys('5164980')
button = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/dl/dt/form/table[1]/tbody/tr[1]/td[2]/input')
button.click()
time.sleep(2)
driver.switch_to.window(driver.window_handles[-1])

In [8]:
# 매물 관리 페이지로 이동
button_2 = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/div[1]/div/ul/li[2]/a')
button_2.click()
time.sleep(2)  # 페이지 로딩 대기

# 테이블이 로드될 때까지 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "bbs_w"))
)

<selenium.webdriver.remote.webelement.WebElement (session="dc16647a906e252388b8e3df87bc8a70", element="f.2B3AB6161470F955CA8546D52EFDF9C5.d.C357FF80595114AE85CA030F210382C8.e.2129")>

In [9]:
property_id = '121527066'

# 1단계: 매물 목록 페이지에서 정보 추출
property_data = {
    "생성일자": datetime.now(),
    "매물번호": property_id,
}
        
        # 매물 목록 페이지에서 필요한 정보 추출
                # 매물 목록 페이지에서 해당 매물 행 찾기
row = driver.find_element(By.XPATH, f"//tr[contains(.//a, '{property_id}')]")
        
# 필요한 정보 추출
property_data.update({
    "거래종류": row.find_element(By.XPATH, "./td[3]").text.strip(),
    "매물종류": row.find_element(By.XPATH, "./td[4]").text.strip(),
    "소재지": row.find_element(By.XPATH, "./td[5]").text.strip(),
    "단지명": row.find_element(By.XPATH, "./td[6]").text.strip(),
    "세부단지명": row.find_element(By.XPATH, "./td[7]").text.strip(),
    "면적": '/'.join(area + "㎡" for area in row.find_element(By.XPATH, "./td[8]").text.strip().split('\n')),
    "가격": row.find_element(By.XPATH, "./td[9]").text.strip(),
})

print(property_data)

{'생성일자': datetime.datetime(2024, 12, 23, 1, 11, 3, 430457), '매물번호': '121527066', '거래종류': '매매', '매물종류': '오피스텔', '소재지': '강남구\n논현동', '단지명': '상지카일룸M', '세부단지명': '1동 705호', '면적': '118.88B㎡/64.95㎡', '가격': '218,000'}


In [10]:
# 수정 버튼 찾아서 클릭
modify_button = driver.find_element(
    By.XPATH, 
    f"//tr[contains(.//a, '{property_id}')]//a[contains(@href, 'goModify')]"
)
modify_button.click()
time.sleep(2)

# 새 창으로 전환
driver.switch_to.window(driver.window_handles[-1])
time.sleep(2)

In [11]:
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')

# 상세 필드 정보 추출
property_fields = {
    "해당층/총층": "해당층/총층", 
    "방향": "방향",
    "현관구조": "현관구조",
    "방수": "방수",
    "총세대수": "총세대수",
    "욕실수": "욕실수",
    "입주가능일": "입주가능일",
    "총 주차대수": "총 주차대수",
    "세대당 주차대수": "세대당 주차대수",
    "해당면적세대수": "해당면적세대수",
    "건축물용도": "건축물용도",
    "매물특징": "매물특징",
    "상세설명": "상세설명"
}

for key, field in property_fields.items():
    try:
        # 모든 th 태그를 찾아서 해당 필드가 포함된 것을 찾기
        for th in soup.find_all('th'):
            if field in th.text:
                td = th.find_next('td')
                
                # 입주가능일 특별 처리
                if field == "입주가능일":
                    # 즉시입주 체크 여부 확인
                    if td.find('input', {'id': 'move_gbn_A', 'checked': True}):
                        field_value = "즉시입주"
                    #  날짜 선택 확인
                    else:
                        year = td.find('select', {'id': 'move_year'}).find('option', selected=True)
                        month = td.find('select', {'id': 'move_month'}).find('option', selected=True)
                        day = td.find('select', {'id': 'move_day'}).find('option', selected=True)
                        
                        if year and month and day:
                            field_value = f"{year.text}/{month.text}/{day.text}이후"
                            
                    # 협의가능 체크 여부 확인
                    if td.find('input', {'id': 'is_move_consult', 'checked': True}):
                        field_value = f"{field_value}(협의가능)" if field_value else "협의가능"
                
                # textarea 태그 확인
                elif td.find('textarea'):
                    field_value = td.find('textarea').text.strip()
                
                # input 태그 확인
                elif td.find('input', {'class': 'input'}):
                    input_tag = td.find('input', {'class': 'input'})
                    if input_tag.get('value'):
                        field_value = input_tag.get('value').strip()
                
                # select 태그 확인
                elif td.find('select'):
                    selected_option = td.find('select').find('option', selected=True)
                    field_value = selected_option.text.strip() if selected_option else None
                
                # 모두 해당 없는 경우 일반 텍스트
                else:
                    field_value = td.get_text(strip=True)
                    
                property_data[key] = field_value
                print(f"[크롤링 데이터] {key}: {field_value}")
                break
                
    except Exception as e:
        print(f"[크롤링 실패] {field} 정보 추출 실패: {str(e)}")
        property_data[key] = None

# 전체 데이터 출력
print("\n=== 전체 크롤링 데이터 ===")
for key, value in property_data.items():
    print(f"{key}: {value}")
print("========================\n")

[크롤링 데이터] 해당층/총층: 7 층
          /
          17 층
[크롤링 데이터] 방향: 거실
[크롤링 데이터] 현관구조: 계단식
[크롤링 데이터] 방수: 1
[크롤링 데이터] 총세대수: 88
[크롤링 데이터] 욕실수: 1
[크롤링 데이터] 입주가능일: 즉시입주(협의가능)
[크롤링 데이터] 총 주차대수: 즉시입주(협의가능)
[크롤링 데이터] 세대당 주차대수: 즉시입주(협의가능)
[크롤링 데이터] 건축물용도: 공동주택
[크롤링 데이터] 매물특징: 강남구청역 도보1분 초역세권 풀옵션 신축 럭셔리 오피스텔 탁트인전망
[크롤링 데이터] 상세설명: 넓은실내공간 양창으로 뷰좋은 호실
강남구청역 더블역세권
사통팔달 교통요지
직주근접 직장인 선호
코엑스개발 최대수혜

전화문의환영합니다!!!
현장매물안내 상담 환영합니다!!!

=== 전체 크롤링 데이터 ===
생성일자: 2024-12-23 01:11:03.430457
매물번호: 121527066
거래종류: 매매
매물종류: 오피스텔
소재지: 강남구
논현동
단지명: 상지카일룸M
세부단지명: 1동 705호
면적: 118.88B㎡/64.95㎡
가격: 218,000
해당층/총층: 7 층
          /
          17 층
방향: 거실
현관구조: 계단식
방수: 1
총세대수: 88
욕실수: 1
입주가능일: 즉시입주(협의가능)
총 주차대수: 즉시입주(협의가능)
세대당 주차대수: 즉시입주(협의가능)
건축물용도: 공동주택
매물특징: 강남구청역 도보1분 초역세권 풀옵션 신축 럭셔리 오피스텔 탁트인전망
상세설명: 넓은실내공간 양창으로 뷰좋은 호실
강남구청역 더블역세권
사통팔달 교통요지
직주근접 직장인 선호
코엑스개발 최대수혜

전화문의환영합니다!!!
현장매물안내 상담 환영합니다!!!



In [12]:
print(property_data['해당층/총층'])

7 층
          /
          17 층


In [13]:
# 모든 데이터 수집 후
if '해당층/총층' in property_data:
    floors = property_data['해당층/총층'].replace('\n', '').replace(' ', '').replace('\xa0', '').split('/')
    floors = [f.strip() for f in floors if f.strip()]
    if len(floors) == 2:
        property_data['해당층/총층'] = f"{floors[0]}/{floors[1]}"

# 4단계: DB 저장
try:
    complex_name = property_data['단지명']
    apt_collection = db['apt']
    
    # property_data에서 단지명 제거 (복사본 생성)
    property_data_to_save = property_data.copy()
    property_data_to_save.pop('단지명')
    
    existing_complex = apt_collection.find_one({
        "단지명": complex_name,
        "매물목록.매물번호": property_id
    })
    
    if existing_complex:
        result = apt_collection.update_one(
            {
                "단지명": complex_name,
                "매물목록.매물번호": property_id
            },
            {"$set": {"매물목록.$": property_data_to_save}}
        )
        print(f"기존 매물 정보 업데이트: {complex_name} - {property_id}")
    else:
        result = apt_collection.update_one(
            {"단지명": complex_name},
            {
                "$push": {"매물목록": property_data_to_save},
                "$setOnInsert": {"이미지목록": []}
            },
            upsert=True
        )
        print(f"새로운 매물 추가: {complex_name} - {property_id}")

except Exception as e:
    print(f"매물 크롤링 중 오류 발생: {str(e)}")

기존 매물 정보 업데이트: 상지카일룸M - 121527066


In [14]:
driver.back()

In [15]:
report_link = driver.find_element(By.XPATH, f"//td[contains(.//a, '{property_id}')]//a[contains(@href, 'ReportGbn')]")
report_link.click()
time.sleep(2)

In [10]:
# 매물 종류 확인
check_property_type(driver)

시도 실패: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[1]/span[2]"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6411C6CC5+28821]
	(No symbol) [0x00007FF641133850]
	(No symbol) [0x00007FF640FD578A]
	(No symbol) [0x00007FF6410291BE]
	(No symbol) [0x00007FF6410294AC]
	(No symbol) [0x00007FF641072647]
	(No symbol) [0x00007FF64104F33F]
	(No symbol) [0x00007FF64106F412]
	(No symbol) [0x00007FF64104F0A3]
	(No symbol) [0x00007FF64101A778]
	(No symbol) [0x00007FF64101B8E1]
	GetHandleVerifier [0x00007FF6414FFCCD+3408029]
	GetHandleVerifier [0x00007FF64151743F+3504143]
	GetHandleVerifier [0x00007FF64150B61D+3455469]
	GetHandleVerifier [0x00007FF64128BDCB+835995]
	(No symbol) [0x00007FF64113EB6F]
	(No symbol) [0x00007FF64113A824

In [72]:
# 데이터 크롤링
process_property_info(driver, property_id)

NameError: name 'process_property_info' is not defined

In [18]:
property_id = '2462850030'
save_path = r"C:\Users\pqstv\Desktop\real_estate_automization\images"

# 이미지 다운로드
download_images(driver, property_id, save_path)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="summaryInfo"]/div[2]/div[2]/button[1]"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6411C6CC5+28821]
	(No symbol) [0x00007FF641133850]
	(No symbol) [0x00007FF640FD578A]
	(No symbol) [0x00007FF6410291BE]
	(No symbol) [0x00007FF6410294AC]
	(No symbol) [0x00007FF641072647]
	(No symbol) [0x00007FF64104F33F]
	(No symbol) [0x00007FF64106F412]
	(No symbol) [0x00007FF64104F0A3]
	(No symbol) [0x00007FF64101A778]
	(No symbol) [0x00007FF64101B8E1]
	GetHandleVerifier [0x00007FF6414FFCCD+3408029]
	GetHandleVerifier [0x00007FF64151743F+3504143]
	GetHandleVerifier [0x00007FF64150B61D+3455469]
	GetHandleVerifier [0x00007FF64128BDCB+835995]
	(No symbol) [0x00007FF64113EB6F]
	(No symbol) [0x00007FF64113A824]
	(

In [311]:
# complexes 컬렉션 조회
print("\n=== 단지 정보 (complexes) ===")
complexes = list(db.complexes.find())  # 전체 데이터를 리스트로 변환
for complex_data in complexes:
    print("\n단지명:", complex_data['단지명'])
    
    print("\n이미지 목록:")
    for img in complex_data.get('이미지목록', []):
        print(f"- 카테고리: {img['카테고리']}")
        print(f"  저장경로: {img['저장경로']}")
    
    print("\n매물 목록:")
    for property in complex_data.get('매물목록', []):
        print(f"\n- 매물번호: {property['매물번호']}")
        print(f"  거래종류: {property.get('거래종류', 'N/A')}")
        print(f"  가격: {property.get('가격', 'N/A')}")
        print(f"  매물특징: {property.get('매물특징', 'N/A')}")
        print(f"  공급/전용면적: {property.get('공급/전용면적', 'N/A')}")
    print("="*50)


=== 단지 정보 (complexes) ===

단지명: 삼익

이미지 목록:
- 카테고리: MAPS
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\MAPS_1.jpg
- 카테고리: BILDLINE
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\BILDLINE_1.jpg
- 카테고리: GRDPLAN
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\GRDPLAN_1.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_1.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_2.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_3.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_4.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_5.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_6.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Desktop\real_estate_automization\images/삼익\HSCP_7.jpg
- 카테고리: HSCP
  저장경로: C:\Users\pqstv\Deskt